# Files & imports

In [1]:
from pandas import read_csv, DataFrame
import pandas as pd

In [2]:
import os
print(os.getcwd())

/Users/cozmaeug/Private/IST PG - DS/DaSH ENG/ist_dash_2024_rec/non_supervised_analysis/notebooks/dataset_2


In [3]:
## https://www.kaggle.com/datasets/hosubjeong/bakery-sales 

bakery_sales = pd.read_csv("/Users/cozmaeug/Private/IST PG - DS/DaSH ENG/ist_dash_2024_rec/non_supervised_analysis/notebooks/dataset_2/dataset/Bakery Sales.csv") 
data_price = pd.read_csv("/Users/cozmaeug/Private/IST PG - DS/DaSH ENG/ist_dash_2024_rec/non_supervised_analysis/notebooks/dataset_2/dataset/Bakery price.csv")

file_tag = "Bakery"


In [4]:
bakery_sales = bakery_sales.drop(['croque monsieur', 'mad garlic'], axis=1)


In [5]:
bakery_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   datetime            2421 non-null   object 
 1   day of week         2420 non-null   object 
 2   total               2420 non-null   float64
 3   place               2166 non-null   object 
 4   angbutter           1973 non-null   float64
 5   plain bread         857 non-null    float64
 6   jam                 220 non-null    float64
 7   americano           412 non-null    float64
 8   croissant           747 non-null    float64
 9   caffe latte         193 non-null    float64
 10  tiramisu croissant  779 non-null    float64
 11  cacao deep          323 non-null    float64
 12  pain au chocolat    587 non-null    float64
 13  almond croissant    202 non-null    float64
 14  milk tea            137 non-null    float64
 15  gateau chocolat     196 non-null    float64
 16  pandor

In [6]:
data_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    22 non-null     object
 1   price   22 non-null     object
dtypes: object(2)
memory usage: 484.0+ bytes


In [7]:
bakery_sales['datetime'] = pd.to_datetime(bakery_sales['datetime'], errors='coerce')


In [8]:
bakery_sales['datetime'].dtype


dtype('<M8[ns]')

In [9]:
bakery_sales['datetime'].isnull().sum()


233

# Fill null values from products with 0

In [10]:
produto_cols = [col for col in bakery_sales.columns if col in ["angbutter", "plain bread", "jam", "americano", "croissant", 
                                                     "caffe latte", "tiramisu croissant", "cacao deep", 
                                                     "pain au chocolat","almond croissant", "croque monsieur", 
                                                       "mad garlic", "milk tea", "gateau chocolat", "pandoro", "cheese cake",
                                                         "lemon ade", "orange pound", "wiener", "vanila latte", "berry ade",
                                                           "tiramisu", "merinque cookies"]]
bakery_sales[produto_cols] = bakery_sales[produto_cols].fillna(0)


# Create additional columns for clustering

Clustering: olhar p fatura e ver o comportamento de compra
ARM: olhar para categorias/produtos mais associados entre si.

Additional Date features

In [11]:
# Split the 'date'' column to have more information and detail on the:
# - day_of_month
# - day_ok_week 
# - is_weekend
# - moment_of_day
# - hour
# - min
#As the dataset only has data from sepecific month of 2021, we don't need to get the year, quarter.

# Create new columns based on the date column 'event_time'
bakery_sales['day_of_year'] = bakery_sales['datetime'].dt.dayofyear  # Day of the year
bakery_sales['week_of_month'] = bakery_sales['datetime'].apply(lambda x: (x.day - 1) // 7 + 1)
bakery_sales['is_weekend'] = bakery_sales['datetime'].dt.weekday.apply(lambda x: True if x >= 5 else False)
bakery_sales['day_of_week_nr'] = bakery_sales['datetime'].dt.weekday  # Monday=0, Sunday=6
bakery_sales['day_of_month'] = bakery_sales['datetime'].dt.day
bakery_sales['month'] = bakery_sales['datetime'].dt.month_name()
bakery_sales['hour'] = bakery_sales['datetime'].dt.hour
bakery_sales['min'] = bakery_sales['datetime'].dt.minute

# Create function to get time of the day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

bakery_sales['time_of_day'] = bakery_sales['hour'].apply(categorize_time_of_day)

# Display the updated DataFrame
bakery_sales.head()

,datetime,day of week,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,...,merinque cookies,day_of_year,week_of_month,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day
0,2019-07-11 15:35:00,Thur,23800.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,192.0,2.0,False,3.0,11.0,July,15.0,35.0,Afternoon
1,2019-07-11 16:10:00,Thur,15800.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,192.0,2.0,False,3.0,11.0,July,16.0,10.0,Afternoon
2,2019-07-12 11:49:00,Fri,58000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,193.0,2.0,False,4.0,12.0,July,11.0,49.0,Morning
3,2019-07-13 13:19:00,Sat,14800.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,194.0,2.0,True,5.0,13.0,July,13.0,19.0,Afternoon
4,2019-07-13 13:22:00,Sat,15600.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,194.0,2.0,True,5.0,13.0,July,13.0,22.0,Afternoon


In [12]:
import pandas as pd

# Assuming your 'month' column is in full text (e.g., 'July', 'January', etc.)
# First, convert the month column to datetime
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}
bakery_sales['month'] = bakery_sales['month'].map(month_map)

# Now, create the season column based on the month
def get_season(month):
    if month in [12, 1, 2]:  # Winter months
        return 'Winter'
    elif month in [3, 4, 5]:  # Spring months
        return 'Spring'
    elif month in [6, 7, 8]:  # Summer months
        return 'Summer'
    else:  # Fall months
        return 'Fall'

bakery_sales['season'] = bakery_sales['month'].apply(get_season)

# Show the resulting DataFrame with the 'season' column
print(bakery_sales[['month', 'season']].head())


   month  season
0    7.0  Summer
1    7.0  Summer
2    7.0  Summer
3    7.0  Summer
4    7.0  Summer


In [13]:
bakery_sales.dropna(subset=['datetime'], inplace=True)


In [14]:
bakery_sales = bakery_sales.drop(columns=["datetime", "day of week"])

In [15]:
bakery_sales.head()

,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,tiramisu croissant,cacao deep,...,day_of_year,week_of_month,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day,season
0,23800.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,...,192.0,2.0,False,3.0,11.0,7.0,15.0,35.0,Afternoon,Summer
1,15800.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,192.0,2.0,False,3.0,11.0,7.0,16.0,10.0,Afternoon,Summer
2,58000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,...,193.0,2.0,False,4.0,12.0,7.0,11.0,49.0,Morning,Summer
3,14800.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,194.0,2.0,True,5.0,13.0,7.0,13.0,19.0,Afternoon,Summer
4,15600.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,194.0,2.0,True,5.0,13.0,7.0,13.0,22.0,Afternoon,Summer


## Feature engineering
- ✅ total = medio alto baixo
- ✅ is food / is drink binary
- cold drink hot drink
- ✅ sazonalidade - winter autumn fall summer 
- how many products per fatura
- how many of each per fatura: X drinks, Y bakery items


In [16]:
bakery_sales['Purchase value'] = pd.qcut(bakery_sales['total'], q=3, 
                                         labels=['Low', 'Medium', 'High'])


In [17]:
# Define product categories
food_items = ['angbutter', 'plain bread', 'jam', 'croissant', "tiramisu croissant", "cacao deep", "pain au chocolat",
              "almond croissant", "gateau chocolat", "pandoro", "cheese cake",
              "orange pound", "wiener", "tiramisu", "merinque cookies"]  
drink_items = ['americano', 'caffe latte', "milk tea", "lemon ade", "vanila latte", "berry ade"]     

# Create the 'product_category' column
# 1 if any food item is sold (any of the food items column has a 1)
# 0 if only drink items are sold (no food items present)
# 3 if both food and drink are sold

bakery_sales['product_category'] = bakery_sales.apply(
    lambda row: 3 if row[food_items].sum() > 0 and row[drink_items].sum() > 0 else 
                (1 if row[food_items].sum() > 0 else 0), axis=1
)
print(bakery_sales['product_category'].value_counts())

product_category
1    1551
3     866
0       4
Name: count, dtype: int64


In [18]:
bakery_sales.head()

,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,tiramisu croissant,cacao deep,...,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day,season,Purchase value,product_category
0,23800.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,...,False,3.0,11.0,7.0,15.0,35.0,Afternoon,Summer,High,3
1,15800.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,3.0,11.0,7.0,16.0,10.0,Afternoon,Summer,Low,1
2,58000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,...,False,4.0,12.0,7.0,11.0,49.0,Morning,Summer,High,1
3,14800.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,5.0,13.0,7.0,13.0,19.0,Afternoon,Summer,Low,3
4,15600.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,True,5.0,13.0,7.0,13.0,22.0,Afternoon,Summer,Low,1


# Save df from preprocessing into a csv file

In [19]:
# Definir o caminho para salvar o arquivo CSV
filepath = r'/Users/cozmaeug/Private/IST PG - DS/DaSH ENG/ist_dash_2024_rec/non_supervised_analysis/notebooks/dataset_2/df_bakery_pre_proc.csv'

# Salvar o DataFrame em CSV
bakery_sales.to_csv(filepath, index=False)

bakery_sales.head()

,total,place,angbutter,plain bread,jam,americano,croissant,caffe latte,tiramisu croissant,cacao deep,...,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day,season,Purchase value,product_category
0,23800.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,...,False,3.0,11.0,7.0,15.0,35.0,Afternoon,Summer,High,3
1,15800.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,3.0,11.0,7.0,16.0,10.0,Afternoon,Summer,Low,1
2,58000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,...,False,4.0,12.0,7.0,11.0,49.0,Morning,Summer,High,1
3,14800.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,5.0,13.0,7.0,13.0,19.0,Afternoon,Summer,Low,3
4,15600.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,True,5.0,13.0,7.0,13.0,22.0,Afternoon,Summer,Low,1


In [20]:
from pandas import read_csv, DataFrame
import pandas as pd
#get file from data pre processing notebook. This file has all the column we need to start the data profiling phase
filepath = r'/Users/cozmaeug/Private/IST PG - DS/DaSH ENG/ist_dash_2024_rec/non_supervised_analysis/notebooks/dataset_2/df_bakery_pre_proc.csv'

file_tag = 'Bakery'

bakery_sales = pd.read_csv(filepath)